In [1]:
!pip install flask_ngrok
!pip install flask
!pip install openai

In [2]:
!pip install python-dotenv

In [9]:
from dotenv import load_dotenv
from pathlib import Path
import os
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

True

In [4]:
from flask import Flask
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)
@app.route("/<name>")  
def home(name):
    return f"<h1>hello {name}</h1>"
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread Thread-13:
Traceback (most recent call last):
  File "C:\Users\jason\anaconda3\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\jason\anaconda3\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\jason\anaconda3\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 無法連線，因為目標電腦拒絕連線。

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\jason\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\jason\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 398, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "C:\Users

In [5]:
!pip install line-bot-sdk

In [8]:
import openai
from flask import Flask, request

# 載入 LINE Message API 相關函式庫
from linebot import LineBotApi, WebhookHandler
from linebot.models import TextSendMessage   # 載入 TextSendMessage 模組
import json

app = Flask(__name__)

@app.route("/", methods=['POST'])
def linebot():
    body = request.get_data(as_text=True)
    json_data = json.loads(body)
    print(json_data)
    try:
#         取得環境變數
        gpt_key=os.environ.get('CHATGPT')
        channel_token=os.environ.get('CHANNEL_TOKEN')
        channel_secret=os.environ.get('CHANNEL_SECRET')
        line_bot_api=LineBotApi(channel_token)
        handler=WebhookHandler(channel_secret)
        signature=request.headers['X-Line-Signature']
        handler.handle(body, signature)
        tk=json_data['events'][0]['replyToken']
        msg=json_data['events'][0]['message']['text']
        # 取出文字的前五個字元，轉換成小寫
        ai_msg = msg[:5].lower()
        reply_msg = ''
        # 取出文字的前五個字元是 chat:
        if ai_msg == 'chat:':
            openai.api_key = gpt_key
            # 將第五個字元之後的訊息發送給 OpenAI
            response = openai.Completion.create(
                model='text-davinci-003',
                prompt=msg[5:],
                max_tokens=256,
                temperature=0.5,
                )
            # 接收到回覆訊息後，移除換行符號
            reply_msg = response["choices"][0]["text"].replace('\n','')
        else:
            reply_msg = msg
        text_message = TextSendMessage(text=reply_msg)
        line_bot_api.reply_message(tk,text_message)
    except:
        print('error')
    return 'OK'

if __name__ == "__main__":
    app.run()  #最後生成網址，貼到line: messaging API-->webhook url

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


{'destination': 'Ue73009b34b87bd9fc36ee196aa196265', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '457193579959025841', 'text': 'chat:中國最高峰是？'}, 'webhookEventId': '01H1K1CFHX2T8Q2123MVYYE3A1', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1685339913280, 'source': {'type': 'user', 'userId': 'U03bbae7973739f2fe42eed377fd32992'}, 'replyToken': '74d08f4f1c3944b195fa44471cdd8aa9', 'mode': 'active'}]}


127.0.0.1 - - [29/May/2023 14:01:16] "POST / HTTP/1.1" 200 -
